<a href="https://colab.research.google.com/github/uarman29/CSC-448-AI/blob/master/Assignment%203/Arman_Uddin_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
# My Awesome Machine Learning Notebook

## Abstract

**Requirements:**
+ Talk about what datasets you've selected for your notebook
+ Include why you choose these datasets
+ Talk about the classification algorithm you plan to use
+ Mention what you hope to find over the course of this notebook

I like ice-cream so I found a dataset on ice-cream and decided to do a linear regression to predict the amount of revenue you would gain on a given day provided the temperature.

For the classification I thought it would be cool if I could categorize people as high risk or not high risk for stroke based on certain attributes so I decided to do just that.

## Section 1: Prepping the Regression datasets
**Requirements:**
+ At a high level, discuss what columns are included in the data
+ Load the dataset you've selected using pandas
+ Show the **"head"** of the data
+ For the columns you will be using in your regression, describe their setup in more detail
+ Clean the data if needed using pandas

The dataset contains two columns, one for the temperature on a given day and one for the revenue on that same day.

Now it's time to use pandas to load and clean up my data.

In [ ]:
# Use pandas to load your dataset and output the first 15 rows
import pandas as pd

ice_cream = pd.read_csv("https://raw.githubusercontent.com/uarman29/CSC-448-AI/master/Assignment%203/data/ice-cream-data.csv")
ice_cream.head(15)

Let me give a bit more detail on my columns:

1. **Temperature** - Measures the average temperature on a given day in Celsius
2. **Revenue** - What we're trying to predict, this is how much revenue, in dollars was gained on a given day

Now while this data is great and all, I dont really like the temperature being measured in Celsius. There is nothing wrong with using Celsius, but it is a little too difficult for my American brain to understand so let's convert it to Fahrenheit.
    

In [ ]:
# Use pandas to clean out data cells that have invalid values for columns of interest

def convert_c_to_f(x):
    x = x * 1.8 + 32
    return float(x)

ice_cream["Temperature"] = ice_cream["Temperature"].apply(convert_c_to_f)

ice_cream.head(15)

So fresh and so clean!

## Section 2: Linear Regression
**Requirements:**
+ Split your cleaned dataset using sklearn into a training and test set
+ Fit a linear regression to your training set
+ Visualize the regressor using matplotlib
+ Report on the error rate for your test set
+ Comment on the result of this regression. Did it work well or not? Provide justification why or why not.

Normally we would extract features here but since we only have two columns there is nothing to extract.

Now we can use sklearn to create a training set and test set. I will reserve 20% of the dataset for testing.

In [ ]:
# Use sklearn to split our dataset
import numpy as np
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(ice_cream["Temperature"], ice_cream["Revenue"], test_size = .2)

Let's use that training set and fit a Linear Regression to it:

In [ ]:
# Use sklearn to create and Linear Regressor and fit our data to it

from sklearn import linear_model

reg = linear_model.LinearRegression()

reg.fit(x_train.values.reshape(-1,1), y_train)

To make sure we're on the right track let's try a couple made up sample inputs and see what our model returns:

In [ ]:
# experiment with the regressor

print("If it was 100 degress F we predict $" + str(round(reg.predict([[100]])[0], 2)))
print("If it was 75 degress F we predict $" + str(round(reg.predict([[75]])[0], 2)))
print("If it was 50 degress F we predict $" + str(round(reg.predict([[50]])[0], 2)))
print("If it was 32 degress F we predict $" + str(round(reg.predict([[32]])[0], 2)))

Now lets plot out linear regression. The blue dots represent training data, the green dots represent test data, and the red line is the regression line.

In [ ]:
# Use matplotlib (or other visualization lib) to create a scatter plot and the regression to create a regression line

import matplotlib.pyplot as plt

plt.figure(figsize=(15,10))
plt.xlabel("Temperature (°F)")
plt.ylabel("Revenue (USD)")
plt.scatter(x_train, y_train, color='blue')
plt.scatter(x_test, y_test, color='green')
plt.plot(x_train, reg.predict(x_train.values.reshape(-1,1)), color='red')
plt.show()

Very spicy. Now let's see what my awesome ML model predicts for the training set and how well it performed:

In [ ]:
# Use linear regression model to report accuracy/error rate on the test set
from sklearn import metrics

y_predicted = reg.predict(x_test.values.reshape(-1,1))
print("The model has an accuracy of", str(round(metrics.r2_score(y_test, y_predicted) * 100, 2)) + "%")

Awesome! It's possible I just got lucky though. Let me do some further analysis to see how well this works

In [ ]:
# Potentially perform further analysis, one good angle is k-fold cross validation or trying a different regression
from sklearn.model_selection import cross_val_score

scores = cross_val_score(reg, ice_cream["Temperature"].values.reshape(-1,1), ice_cream["Revenue"].values.reshape(-1,1), cv=5)
print("%0.2f%% accuracy from 5-fold cross validation" % (scores.mean() * 100))

So it looks like our model is pretty accurate.

## Section 3: Prepping the classification dataset
**Requirements:**
+ At a high level, discuss what columns are included in the data
+ Load the dataset you've selected using pandas
+ Show the **head** of the data
+ For the columns you will be using in your classification, describe the range of values
+ For those columns map the values to a set of integers (if they aren't already)
+ Clean the data using pandas if needed

For the classification section I choose a dataset that contains health information, including whether or not the person has had a stroke previously. I hope to use the various pieces of health information to train a model that can classify people as high risk or not high risk for stroke. The three columns that I will focus on as as follows:

1. Age - The person's age
2. BMI - The person's body mass index which is derived from a persons height and weight to determine if they are at a heathly weight for their height
3. Sex - possible values: 'Male', 'Female' (Referred to as Gender in data)

I want my classifier to output a 1 for someone who is at a high risk for stroke and a 0 for someone who is not at a high risk.

I will have to clean the data because there are some entries with N/A as a value, and that is something I do not want. There are also some other issues I will take care when cleaning the data.

I also will be mapping Male to 0 and Female to 1.

In [ ]:
# Use pandas to load dataset, clean dataset, and output the first 15 rows
stroke_data = pd.read_csv("https://raw.githubusercontent.com/uarman29/CSC-448-AI/master/Assignment%203/data/stroke-data.csv")
stroke_data = stroke_data[["age", "bmi", "gender", "stroke"]]

def gender_map(x):
    if x == "Male":
        return 0
    if x == "Female":
        return 1

stroke_data["age"] = stroke_data["age"].apply(pd.to_numeric)
stroke_data["bmi"] = stroke_data["bmi"].apply(pd.to_numeric)
stroke_data["gender"] = stroke_data["gender"].apply(gender_map)
stroke_data["gender"] = stroke_data["gender"].apply(pd.to_numeric)
stroke_data = stroke_data.dropna()
stroke_data.head(15)

## Section 4: Classification with a Neural Network (or classifier of choice)
**Requirements:**
+ Split your cleaned dataset using sklearn into a training and test set
+ **(Situational)** Scale your data to prevent overfitting
+ Fit a classifier to your training set
+ **(Optional)** visualize the classifier on a data plot
+ Report on the error rate for your test set
+ Comment on the result of this regression. Did it work well or not? Provide justification why or why not.


Now we can train our model to use age, bmi, and gender to predict if a person is at a high risk for stroke.

Once again let's split out our data into a training and test set. I will hold 20% of the data for testing.

In [ ]:
# Use sklearn to split our dataset
x_train, x_test, y_train, y_test = train_test_split(stroke_data[["age", "bmi", "gender"]], stroke_data["stroke"], test_size = .2)

Before we train we also need to scale our data for our neural network. This is because neural networks are suspetible to overfitting on features that have a higher magnitude. To scale this data I will use a [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

In [ ]:
# Use sklearn to scale your feature vector (if you use a neural network)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

Alright now that our data is ready, let's create our classifier. For this project I will use a SVM.

In [ ]:
# Initialize SVM
from sklearn import svm

clf = svm.SVC()

Now let's train this SVM.

In [ ]:
# Fit the network and do a sample
clf.fit(x_train, y_train)
if clf.predict([[21, 20, 0]])[0] == 0:
    print("A 21 year old male with a BMI of 20 is not at a high risk for stroke")
else:
    print("A 21 year old male with a BMI of 20 is at a high risk for stroke")

Once again let's analyze the accuracy of our machine learning. Here is the accuracy for the test set.

In [ ]:
# Report on the accuracy of the Test set for your classifier

predicted_y = clf.predict(x_test)
print("My model has an accuracy of "+ str(round(metrics.accuracy_score(y_test, predicted_y) * 100, 2)) + "%")

And just as before let's do a k-fold cross validation to see if we didn't just get lucky (or unlucky on our first go).

In [ ]:
# Just as in Section 2, perform more analysis, try different neural network architectures or ways of splitting the data

scores = cross_val_score(clf, stroke_data[["age", "bmi", "gender"]], stroke_data["stroke"], cv=5)
print("%0.2f%% accuracy from 5-fold cross validation" % (scores.mean() * 100))

Based on the results we can see that age, bmi, and gender are a good predictor of whether or not someone is at a high risk for stroke.

## Conclusions
**Requirements:**
+ Summarize the findings of your report
+ Repeat your methodology and key findings for Section 2. and Section 4.
+ Highlight what you found interesting
+ Discuss what you would do to extend the project further


Here I summarize what I did in this report. Highlighting what I did, what I found interesting or surprising. I sum up and conclusions and what I would do to extend this project further in a V2.

In this report I used linear regression to find the correlation between two sets of values, and then I used classification in the form of a Support Vector Machine to classify my data into two groups so that I could predict which group a set of data would belong to. The methodology for Section 2 and Section 4 were almost identical. I started by loading the data from the CSV and then cleaning up missing/invalid data using pandas. Then I split my data into training data and test data. At this point for Section 4 I had to scale my data using MinMaxScaler so that each feature had equal influence. Then I simply trained/fit my respective models using the training data. For Section 2 I also included a graph. Finally, I used my test data to check how accurate each model was and then did cross validation to make sure I didn’t just get lucky. From this report I found out that a day's ice cream revenue is very closely related to the temperature on that day, and I also found out that age, bmi, and gender are great predictors of whether or not someone is at a high risk for stroke. Overall, I thought it was really interesting how accurate these models could be, especially for my classification model. It really got me thinking about how AI is the future and maybe one day we will have robot doctors because my model did a pretty good job of categorizing someone as at a high risk for stroke or not. To extend this project I could do a few things like, add more features to my feature vector to make it more accurate and use different models to test out the effectiveness of each. 